In [21]:
from src.data import load_data, util
from src.models import util_model

In [22]:
image_name = "trans005" 
front_image_name = image_name + "-1"
back_image_name = image_name + "-2"
front_image_path = "data/raw/front"
back_image_path = "data/raw/back"
front_image = load_data.read_image(front_image_path, front_image_name)
back_image = load_data.read_image(back_image_path, back_image_name)

In [23]:
front_section_dict = util_model.detect_section(front_image, util_model.front_model)
pf_sections = util.pre_process(front_section_dict)
front_text_dict = util_model.images_to_texts(pf_sections)
courses_df = util.make_course(front_text_dict)
error_df = util.get_error(courses_df)
error_df


0: 640x480 3 sections, 607.2ms
Speed: 2.1ms preprocess, 607.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)


,ocr,numeric,section,ref,id,name,unit,grade


In [24]:
post_courses_df = util.post_process(courses_df, pf_sections)
post_error_df = util.get_error(post_courses_df).reset_index(drop=True)
post_error_df

,ocr,numeric,section,ref,id,name,unit,grade


In [25]:
post_error_df.ocr.to_list()

[]

In [26]:
back_section_dict = util_model.detect_section(back_image, util_model.back_model)
pb_sections = util.pre_process(back_section_dict)
back_text_dict = util_model.images_to_texts(pb_sections)
gpa = util.get_GPA(back_text_dict[0])
gpa


0: 640x480 1 section, 597.7ms
Speed: 2.3ms preprocess, 597.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)


,ocr,category,unit,grade
0,กก ภาษาไทย 6.0 4.00,ภาษาไทย,6.000,4.0
1,กข คณิตศาสตร์ 18.0 4.00,คณิตศาสตร์,18.000,4.0
2,กค วิทยาศาสตร์และเทคโนโลยี 38.5 4.00,วิทยาศาสตร์,38.500,4.0
3,กญ (สังคมศึกษา ศาสนา และวัฒนธรรม 8.0 4.00,สังคมศึกษา,8.000,4.0
4,กฐ สุขศึกษาและพลศึกษา 3.5 4.00,สุขศึกษา,3.500,4.0
5,กท ศิลปะ 3.0 4.00,ศิลปะ,3.000,4.0
6,กย การงานอาชีพ 3.0 4.00,การงานอาชีพ,3.000,4.0
7,กพ ภาษาต่างประเทศ 11.5 4.00,ภาษาต่างประเทศ,11.500,4.0
8,กว การศึกษาค้นคว้าด้วยตนเอง (!5) 2.0 4.00,ศึกษาค้นคว้าด้วยตนเอง,2.000,4.0
9,กอ ผลการเรียนเฉลี่ยตลอดหลักสูตร 93.5 4.00,ผลการเรียน,0.935,4.0


In [27]:
front_edf = post_courses_df[["section", "id", "name", "unit", "grade"]]
back_edf = gpa[["category", "unit", "grade"]]

In [28]:
front_edf

,section,id,name,unit,grade
0,000,ท33101,ภาษาไทยพื่นฐาน,1.0,4.0
1,001,ส33101,สังคมศึกษาพื้นฐาน,1.0,4.0
2,002,ส33103,ประวัติศาสตร์,0.5,4.0
3,003,พ33101,สุขศึกษาพลศึกษา,0.5,4.0
4,004,ศ33101,ศิลปะ,0.5,4.0
...,...,...,...,...,...
85,227,ว32252,โปรแกรมประมวลผลสารสนเทศ,1.0,4.0
86,228,ง32221,กระบวนการงานอาชีพ,0.5,4.0
87,229,จ32206,สนทนาภาษาจีน,0.5,4.0
88,230,อ32202,สนทนาภาษาอังกฤษ,0.5,4.0


In [29]:
back_edf

,category,unit,grade
0,ภาษาไทย,6.000,4.0
1,คณิตศาสตร์,18.000,4.0
2,วิทยาศาสตร์,38.500,4.0
3,สังคมศึกษา,8.000,4.0
4,สุขศึกษา,3.500,4.0
5,ศิลปะ,3.000,4.0
6,การงานอาชีพ,3.000,4.0
7,ภาษาต่างประเทศ,11.500,4.0
8,ศึกษาค้นคว้าด้วยตนเอง,2.000,4.0
9,ผลการเรียน,0.935,4.0


In [30]:
load_data.export_df_to_csv("data/out/csv/front", front_image_name, front_edf)
load_data.csv_to_json("data/out/json/front", "data/out/csv/front", front_image_name)
load_data.export_df_to_csv("data/out/csv/back", back_image_name, back_edf)
load_data.csv_to_json("data/out/json/back", "data/out/csv/back", back_image_name)